In [2]:
import pandas as pd
import pulp
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
# COAST, NCENT, NORTH, SCENT, SOUTH, WEST, EAST, FWEST
REGION = 'FWEST'
RATING = 500
CAPACITY = 700

In [4]:
def pulp24hrBattery(pred_load, actual_load, RATING, CAPACITY):
    # A pulp LP optimizer of 24 hr battery
    model = pulp.LpProblem("Daily demand charge minimization problem", pulp.LpMinimize)
    power = pulp.LpVariable.dicts("ChargingPower", range(24))

    for i in range(24):
        power[i].lowBound = 0
        power[i].upBound = RATING
    pDemand = pulp.LpVariable("Peak Demand", lowBound=0)

    model += pDemand

    for i in range(24):
        model += pDemand >= pred_load[i] - power[i]    
    model += pulp.lpSum(power) <= CAPACITY
    
    model.solve()
    return [actual_load[i] - power[i].varValue for i in range(24)]

def optimal_daily(day_load, RATING, CAPACITY):
    ps = max(day_load)
    new_load = list(day_load)
    broken = True
    while broken:
        ps -= 1
        new_load = [ps if l > ps else l for l in new_load]
        diff = [p - l for p, l in zip(day_load, new_load)]
        broken = sum(diff) <= CAPACITY and all([d <= RATING for d in diff])    
    return new_load

In [1]:
d = {}

REGIONS = ['COAST', 'NCENT', 'NORTH', 'SCENT', 'SOUTH', 'WEST', 'EAST', 'FWEST']

for region in REGIONS:
    d[region] = {}
    df = pd.read_csv('./data/' + region + '.csv', parse_dates={'dates': [2,3,4]})
    df = df.drop('tempc', axis=1)
    
    m = df['load'].tolist()
    load_24 = [m[i: i+24] for i in range(0, len(m), 24)]
    d[region]['max_load'] = df.groupby(df.dates.dt.date)['load'].max().sum()
    for error in range(0, 6):
        error = float(error) / 100
        
        df['prediction'] = df['load'].apply(lambda x: x + np.random.normal(0, error*x))
        m = df['prediction'].tolist()
        pred_24 = [m[i: i+24] for i in range(0, len(m), 24)]
        
        n_load = []
        for pred_load, actual_load in zip(pred_24, load_24):
            n_load.append(pulp24hrBattery(pred_load, actual_load, RATING, CAPACITY))
        
        NEW_MAX_LOAD = sum([max(d) for d in n_load])
        
        d[region][str(error)] = NEW_MAX_LOAD

NameError: name 'pd' is not defined